In [2]:
import json
import pandas as pd
import os
import openai
from dotenv import load_dotenv

In [9]:
# Load environment variables (API key)
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

file_path = "Given\Book.xlsx"
excel_file_path = "Given\Book.xlsx"

<>:5: SyntaxWarning: invalid escape sequence '\B'
<>:6: SyntaxWarning: invalid escape sequence '\B'
<>:5: SyntaxWarning: invalid escape sequence '\B'
<>:6: SyntaxWarning: invalid escape sequence '\B'
C:\Users\Sarthak Jain\AppData\Local\Temp\ipykernel_5492\3873988671.py:5: SyntaxWarning: invalid escape sequence '\B'
  file_path = "Given\Book.xlsx"
C:\Users\Sarthak Jain\AppData\Local\Temp\ipykernel_5492\3873988671.py:6: SyntaxWarning: invalid escape sequence '\B'
  excel_file_path = "Given\Book.xlsx"


In [10]:
def read_excel_to_string(file_path):
    """Read Excel file and convert to a string representation"""
    df = pd.read_excel(file_path)
    return df.to_string(index=False)

def format_po_data(po_data):
    """Format PO data into a string representation"""
    result = ""
    for po_num, lines in po_data.items():
        result += f"PO No. {po_num}\n"
        result += "Line\tPart Number\tOrdered Quantity\n"
        for line in lines:
            result += f"{line['line']}\t{line['part_number']}\t{line['ordered_qty']}\n"
        result += "\n"
    return result

In [11]:
def process_shipping_report(excel_file_path, po_data):
    """Process shipping report using GPT-4"""
    
    # Read shipping report from Excel
    shipping_report = read_excel_to_string(excel_file_path)
    
    # Format PO data
    formatted_po_data = format_po_data(po_data)
    
    # Create prompt with actual data
    prompt_template = """
    You are a supply chain data processing assistant specialized in extracting and transforming shipping report data. Your task is to process a supplier's shipping report in relation to existing Purchase Order (PO) data, apply specific business rules, and generate updated PO information.

    ### CONTEXT:
    - You will be provided with a shipping report from a supplier in tabular format
    - You will also be provided with current PO data showing ordered quantities
    - You need to extract relevant data, compare with existing PO information, and apply business rules

    ### SHIPPING REPORT STRUCTURE:
    The shipping report contains the following key columns (among others):
    - PO NUMBER: The purchase order number
    - PART NUMBER: The part/item identifier
    - PO LINE ITEM: The line number on the PO
    - QTY: The quantity shipped in this shipment
    - DATE: The date of shipment
    - TRACKING NUMBER: Shipment tracking identifier
    - Other metadata fields (prices, descriptions, etc.)

    ### CURRENT PO DATA:
    The current PO data shows:
    - PO Number
    - Line Number
    - Part Number
    - Ordered Quantity

    ### BUSINESS RULES TO APPLY:
    1. If the quantity shipped is lower than the ordered quantity for any PO line, create a new line for the remaining quantity still on order.
    2. If the same item is shipped in multiple shipments, split the PO line into multiple release lines.

    ### TASK:
    1. Extract all relevant shipping data from the provided shipping report.
    2. Compare the shipment details with the existing PO lines and quantities.
    3. Apply the business rules to generate updated PO information.
    4. Format the output as a structured JSON object showing all current (updated) PO lines.

    ### OUTPUT FORMAT:
    Provide a JSON object with the following structure:
    - An array of PO objects, each containing:
      - po_number: The PO number
      - lines: An array of line objects, each containing:
        - line_number: The original PO line number
        - release_number: A sequential number for each shipment of the same line
        - part_number: The part number
        - original_ordered_qty: The original ordered quantity
        - shipped_qty: The quantity shipped in this release
        - remaining_qty: The quantity still remaining to be shipped (if any)
        - shipment_date: The date of shipment
        - tracking_number: The tracking number for this shipment
        - status: "Shipped" for shipped items, "On Order" for remaining quantities

    ### EXAMPLE:
    For a PO line with 100 units ordered, where 60 units were shipped in one shipment and 20 in another, with 20 remaining, the output would show three lines: two "Shipped" lines (release 1 with 60 units and release 2 with 20 units) and one "On Order" line (with 20 units remaining).

    Now, process the following data:

    SHIPPING REPORT:
    {shipping_report}

    CURRENT PO DATA:
    {po_data}
    
    Respond with ONLY the JSON output, no additional text.
    """
    
    # Fill in the template
    prompt = prompt_template.format(
        shipping_report=shipping_report,
        po_data=formatted_po_data
    )
    
    # Call GPT-4
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a data processing assistant that extracts structured information from shipping reports."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1,  # Low temperature for more deterministic results
        max_tokens=2000
    )
    
    # Extract and parse the JSON response
    result = response.choices[0].message.content.strip()
    
    # If the response is wrapped in ```json and ```, remove them
    if result.startswith("```json"):
        result = result.replace("```json", "", 1)
    if result.endswith("```"):
        result = result[:-3]
    
    # Parse the JSON
    try:
        parsed_result = json.loads(result.strip())
        return parsed_result
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        print(f"Raw response: {result}")
        return None

In [12]:
# Example usage
if __name__ == "__main__":
    # Sample PO data structure
    po_data = {
        "H006374": [
            {"line": "29", "part_number": "160-1887-1-TUFYH", "ordered_qty": 15},
            {"line": "33", "part_number": "182-837FE-TUFYH", "ordered_qty": 120},
            {"line": "32", "part_number": "182-837ME-TUFYH", "ordered_qty": 20},
            {"line": "25", "part_number": "493-13780-1-TUFYH", "ordered_qty": 300}
        ],
        "SM322693": [
            {"line": "19", "part_number": "541-2.00DCT-TUFYH", "ordered_qty": 55}
        ]
    }
    
    # Process the shipping report
    result = process_shipping_report("Given\Book.xlsx", po_data)
    
    # Save the result to a JSON file
    if result:
        with open("updated_po_data.json", "w") as f:
            json.dump(result, f, indent=2)
        print("Updated PO data saved to updated_po_data.json")

<>:17: SyntaxWarning: invalid escape sequence '\B'
<>:17: SyntaxWarning: invalid escape sequence '\B'
C:\Users\Sarthak Jain\AppData\Local\Temp\ipykernel_5492\2930100657.py:17: SyntaxWarning: invalid escape sequence '\B'
  result = process_shipping_report("Given\Book.xlsx", po_data)
C:\Users\Sarthak Jain\AppData\Local\Temp\ipykernel_5492\2930100657.py:17: SyntaxWarning: invalid escape sequence '\B'
  result = process_shipping_report("Given\Book.xlsx", po_data)


FileNotFoundError: [Errno 2] No such file or directory: 'Given\\Book.xlsx'